### Imports

In [1]:
# General imports
import pandas as pd
import warnings
import logging
import os
from tensorflow import keras
from utils import (move_to_working_folder, preprocess, build_deberta, generate_predictions, seed_everything, CFG)


# Disabling unnecceseray warnings
warnings.simplefilter("ignore")
logging.disable(logging.ERROR)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# seed_everything(random_seed=CFG.random_seed)

2024-07-28 20:12:40.058439: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 20:12:40.058542: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 20:12:40.197010: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Processing /kaggle/input/autocorrect/autocorrect-2.6.1.tar
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622364 sha256=506e43d7ea2b1c152fdd1c2e20e294d1192ec062fd21420d98355110240712cb
  Stored in directory: /root/.cache/pip/wheels/db/69/42/0fb0421d2fe70d195a04665edc760cfe5fd341d7bb8d8e0aaa
Successfully built autocorrect


### Load Model

In [2]:
# Choose the model to load
epochs = 12
CFG.final_model_path = f'full_model_scaled-{epochs}.keras'

In [3]:
# Load the model 
model_path = '/kaggle/working/' + CFG.final_model_path
move_to_working_folder('/kaggle/input/models/' + CFG.final_model_path, model_path)

model = keras.models.load_model(model_path)
deberta = build_deberta()
print(f'{model_path} loaded successfully')

/kaggle/working/full_model_scaled-12.keras loaded successfully


### Sample Random Summary

In [4]:
df = pd.read_excel('/kaggle/input/llm-generate-test/LLM_Generate_Test.xlsx')
random_row = df.sample(n=1)

student_id = random_row.iloc[0,0]
summary = random_row.iloc[0,3]
prompt_question = random_row.iloc[0,2]
prompt_text = random_row.iloc[0,1]

print(f'student_id: {student_id}\n')
print(f'prompt_question: {prompt_question}\n')
print(f'summary: {summary}')

student_id: 16014

prompt_question: Summarize the main causes and effects of the American Civil War. Cite evidence from the text.



summary: The American Civil War was sparked by discrepancies between the North and the South about slavery and states' rights. The war led to the abolition of slavery and strengthening of the federal government, as well as a redefinition of American identity.


### Score The Summary

In [5]:
# This function gets a students id, summary, prompt question and prompt text
# and scores the summary by content and wording
def run_demo(student_id, summary, prompt_question, prompt_text):
    input_ids, attention_mask, head_mask = preprocess(summary, prompt_question, prompt_text, deberta.tokenizer, is_demo=True)
    
    inputs = {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'head_mask': head_mask,
        'student_id': [student_id],
    }
    content_score, wording_score = (generate_predictions(model, inputs))[1:]
    return content_score[0], wording_score[0]

content_score, wording_score = run_demo(student_id, summary, prompt_question, prompt_text)

print(f'Student id: {student_id}')
print(f'Content score: {content_score}')
print(f'Wording score: {wording_score}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
Student id: 16014
Content score: -1.0146484375
Wording score: -1.1611328125
